In [2]:
import os
import re

mmode0 = re.compile(r'\$\$.+?\$\$', flags = re.DOTALL | re.UNICODE)
mmode1 = re.compile(r'\$.+?\$', flags = re.DOTALL | re.UNICODE)
mmode2 = re.compile(r'\\\[.+?\\\]', flags = re.DOTALL | re.UNICODE)
mmode3 = re.compile(r'\\begin{equation}.+?\\end{equation}', flags = re.DOTALL | re.UNICODE)
mmode4 = re.compile(r'\\begin{eqnarray}.+?\\end{eqnarray}', flags = re.DOTALL | re.UNICODE)
mmode5 = re.compile(r'\\begin{equation\*}.+?\\end{equation\*}', flags = re.DOTALL | re.UNICODE)
mmode6 = re.compile(r'\\begin{eqnarray\*}.+?\\end{eqnarray\*}', flags = re.DOTALL | re.UNICODE)

In [3]:
def primary_processing(inp):
    inp = re.sub(r'.*\\newcommand.*', '', inp)
    
    s0 = mmode0.findall(inp)
    inp = mmode0.sub('', inp)

    s1 = mmode1.findall(inp)
    inp = mmode1.sub('', inp)

    s2 = mmode2.findall(inp)
    inp = mmode2.sub('', inp)

    s3 = mmode3.findall(inp)
    inp = mmode3.sub('', inp)

    s4 = mmode4.findall(inp)
    inp = mmode4.sub('', inp)
    
    s5 = mmode5.findall(inp)
    inp = mmode5.sub('', inp)
    
    s6 = mmode6.findall(inp)
    inp = mmode6.sub('', inp)

#     print s0[0:5]
#     print s1[0:5]
#     print s2[0:5]
#     print s3[0:5]
#     print s4[0:5]

    ret = []
    
    [ret.append(i) for i in s0]
    [ret.append(i) for i in s1]
    [ret.append(i) for i in s2]
    [ret.append(i) for i in s3]
    [ret.append(i) for i in s4]
    [ret.append(i) for i in s5]
    [ret.append(i) for i in s6]
    
    return ret

In [4]:
def secondary_processing(inp):
    inp = inp.replace('\n',' ')
    
    if mmode0.match(inp) != None:
        inp = inp.replace('$$','')
    
    if mmode1.match(inp) != None:
        inp = inp.replace('$','')
    
    if mmode2.match(inp) != None:
        inp = inp.replace('\\[','')
        inp = inp.replace('\\]','')
    
    if mmode3.match(inp) != None:
        inp = inp.replace('\\begin{equation}','')
        inp = inp.replace('\\end{equation}','')
    
    if mmode4.match(inp) != None:
        inp = inp.replace('\\begin{eqnarray}','')
        inp = inp.replace('\\end{eqnarray}','')
        
    if mmode5.match(inp) != None:
        inp = inp.replace('\\begin{equation*}','')
        inp = inp.replace('\\end{equation*}','')
    
    if mmode6.match(inp) != None:
        inp = inp.replace('\\begin{eqnarray*}','')
        inp = inp.replace('\\end{eqnarray*}','')
    
    return inp

In [27]:
def tertiary_processing(inp):
    inp = inp.replace('\\nonumber', '')
    inp = re.sub('&', ' ', inp)
    
    inp = inp.split('\\\\')
    ret = []
    for i in inp:
        if i != '\\':
            ret.append(i.strip())
    
    return ret

In [13]:
def full_processing(inp):
    inp = primary_processing(inp)
    inp = [secondary_processing(i) for i in inp]
    
    processed = []
    [processed.extend(tertiary_processing(i)) for i in inp]
    processed = [i for i in processed if len(i) <= 600]
    
    return processed

In [26]:
text = unicode(file('../../Dataset/1992/9211086','r').read(), errors = 'replace')
# out = primary_processing(text)

# # out = [secondary_processing(i) for i in out]

# # for o in out:
# #     print o

# # print mmode4.match(out[-1]).group(0)
# print out[-1]
# print '#'
# print secondary_processing(out[-1])
try:
    out = full_processing(text)
except UnicodeDecodeError as obj:
    print obj
    raise
# print out
for o in out:
    print '#'
    print o
    print '#'

#
#2\eqno(\the\secno.\the\meqno)
#
#
{\cal L} ={1\over 2}\ g_{\mu\nu}(\phi )\ \partial_t\phi^\mu   \partial_t\phi^\nu +{1\over 2}\ k_{\mu\nu}(\phi )\   \partial_i\phi^\mu\partial_i\phi^\nu \eqno(1.1)
#
#
\eqalignno{   g_{\mu\nu}(\phi )  =e_\mu\ ^\alpha (\phi )\ e_\nu\ ^\beta (\phi )\   g_{\alpha\beta}\cr   k_{\mu\nu}(\phi )  =e_\mu\ ^\alpha (\phi )\ e_\nu\ ^\beta (\phi )\   k_{\alpha\beta}               (1.2)\cr}
#
#
\Lambda\geq\vert\ \bp\vert >{\Lambda\over s}\qquad (s>1)
#
#
\eqalign{   g_{\alpha\beta}\to g_{\alpha\beta}  +\Delta g_{\alpha\beta}\cr   k_{\alpha\beta}\to k_{\alpha\beta}  +\Delta k_{\alpha\beta}\ .\cr}
#
#
\phi^\mu (x)=\bar\phi^\mu (x) +\xi^\mu (x) +\dots   \eqno(2.1)
#
#
{\partial^2\highgamma^\mu\over\partial u^2} +   {\partial\highgamma^\lambda\over\partial u}\   {\partial\highgamma^\nu\over\partial u}\   \Gamma_{\nu\lambda}\ ^\mu (\highgamma )=0                 \eqno(2.2)
#
#
\eqalignno{   \highgamma^\mu (x,0)  =\bar\phi^\mu (x)\cr   \highgamma^\mu (x,1)  =\phi^\mu (

In [20]:
def main():
    out = open('../Data/Formulae', 'w')
    metaout = open('../Data/Meta', 'w')
    
    for year in xrange(1992, 2004):
        
        print 'Processing year {0}'.format(year)
        files = os.listdir('../../Dataset/{0}'.format(year))
        
        for afile in files:
            text = open('../../Dataset/{0}/{1}'.format(year, afile), 'r').read()
            try:
                for form in full_processing(text):
                    out.write('{0}\n'.format(form))
                    metaout.write('{0} {1}\n'.format(year, afile))
            except Exception as obj:
                print year, afile
                print obj
                raise

                
    out.close()
    metaout.close()
        
if __name__ == '__main__':
    main()

Processing year 1992
Processing year 1993
Processing year 1994
Processing year 1995
Processing year 1996
Processing year 1997
Processing year 1998
Processing year 1999
Processing year 2000
Processing year 2001
Processing year 2002
Processing year 2003
